# 9. Deploy Media Graph

In this section, we will deploy media graphs onto our IoT Edge device to trigger our modules to perform certain actions.

## 9.1. Deploy / Manage Media Graph on LVA Edge Module
At this stage of the sample, we should have all required modules deployed to the Edge device, up and running.  

Now we will deploy media graphs onto the "lvaEdge" module. Here, "lvaEdge" is the name that we assigned to the LVA module inside the module deployment template json file from the [previous sample](08_deploy_iotedge_modules.ipynb). Media graph is also a Json file that defines a media flow pipeline from media sources to the target sinks and any analytics processes in-between. The media graph file consists of following modules:  

```
{
    "@apiVersion": "1.0",

    "name": "SampleMediaGraphOrganization",

    "properties":{

        "parameters": ...

        "sources": ...

        "processes": ...

        "sinks": ...
    }
}
```

In the media graph Json file above, the "parameters" section is the most critical part of the media graph. The "parameters" section defines all required parameters to run the media graph. In this section, we will define the addresses of the media sources, their access credentials, and the parameters related to the process modules, such as fps rate and sensitivity of motion detection. Each media graph may have a different media flow, i.e., one may have a motion detection processor while another may not. You may modify this media graph according to your needs.  

### 9.1.1. Sample Media Graph
In this sample project, we provide a sample media graph file "GraphTopology_HTTPExtension_Jpeg_01.json".  The media graph in this file ingests a video stream from a single IP camera, runs motion detector processor on the stream, and sends the stream to one of two targets if there is motion. The first target is Azure Media Services, a cloud platform that can archive the stream's motion chunks. The other possible target is the custom AI module that we developed previous sections of this sample; the AI module analyzes the stream (a chunk with motion) and then sends the result (a Json-format inference result) to IoT Hub.

### 9.1.2. Setting Parameters in Media Graph
There are two ways to set parameters in the media graph:  
1. Update the default values of the parameters in "GraphTopology_HTTPExtension_Jpeg_01.json", which is **not** the prefered method. 
2. Note the parameter names and data types in "GraphTopology_HTTPExtension_Jpeg_01.json", and override their values in the deployment command that we will show in the next section.

### 9.1.3. Deploy the Media Graph with Custom Parameters
Below, we will explain the following operations:
1. Setting (deploying) Media Graph Topology on the IoT Edge Device's lvaEdge module
2. Creating a Media Topolgy Instance from the one set in step 1
3. Activate the Topology Instance that set in step 2

We will also show how to
* List and delete the already set Graph Topologies on the Device
* List and delete the already set Graph Instances on the Device
* Acticate / deactivate the Graph Instances

#### 9.1.3.1. Install Prerequisites
We will use a simple Python script to deploy our media graph Json file into the "lvaEdge" module. Our process entails sending a file into the IoT Edge device over the Internet through a IoT Edge Hub service. To do so, we need to install following Python packages. These packages were developed by Microsoft and help with interacting with IoT Hub Services.

!Important: As mentioned in the first section of this tutorial, you must use the right ```pip``` command (```pip``` or ```pip3```) depending on your Python installation. This sample is tested with Python version 3.6 and so we use the ```pip3``` command to install the packages into our Python3 environment.

In [ ]:
!pip3 install azure-iot-device
!pip3 install azure-iot-hub

#### 9.1.3.2. Import A Helper Python Class to Deploy Media Graphs
The following code snippet will import a custom Python class to help us deploy media edge graphs. After the import, we will instantiate a graph manager object with IoT Hub, IoT Edge Device, and Graph API version details.

In [ ]:
from graph_manager import GraphManager

moduleId = "lvaEdge" # Must be same as the name that we assigned to LVA module in the "deployment.lvaedge_isserver_rtspsim.template.json" file
operationsApiVersion = "1.0"  # Must be same as the version number in the "GraphTopology_HTTPExtension_Jpeg_01.json" file

graphManager = GraphManager(iotHubConnString, iotDeviceId, moduleId, operationsApiVersion)

#### 9.1.3.3. List Existing Graph Topologies and Instances


In [ ]:
# List topologies
response = graphManager.GenericCall("GraphTopologyList", {})
print(response)

In [ ]:
# List instances
response = graphManager.GenericCall("GraphInstanceList", {})
print(response)

#### 9.1.3.4. Set Topology
The following code snippet will deploy the "GraphTopology_HTTPExtension_Jpeg_01.json" file into the lvaEdge module running on our IoT Edge device.

In [ ]:
# You can define a topologyFile or a topologyUrl that you want to deploy into the module. Here we point to our sample Media Graph Topology File.
operationParams = {
                    "topologyFile": "GraphTopology_HTTPExtension_Jpeg_01.json"
                    }

In [ ]:
# Set Graph Topology
response = graphManager.GraphTopologySet(operationParams)
print(response)

#### 9.1.3.5. Set Topology Instance

The following code snippet will set the values of the parameters mentined in "GraphTopology_HTTPExtension_Jpeg_01.json" and will create a Topology Instance. If you modify the topology or add new parameters, you should update the parameter list below accordigly.  

>[!IMPORTANT]  
>- Be sure that the value of "topologyName" parameter below and the same parameter in the "GraphTopology_HTTPExtension_Jpeg_01.json" file are the same.  
>- Since we don't have a physical IP camera set for this sample, we use a virtual IP camera simulator. In the previous steps when we deployed the module deployment manifest, we already deployed a simulator module which plays "sample.mkv" video file as if played from an IP camera. Thus, in the parameter list below, the parameter "rtspUrl" points to the URL address of this IoT Module (rtsp://rtspsim:554) with the full path of the video file that we want to play. You can replace this parameter's value with full RTSP address of your phyical IP camera.  
>- rtspUsername and rtspPassword are dummy values for our simulator because they do not require authentication. However, if your source stream (i.e., your phyical IP Camera) requires authentication, then put the appropriate values for these parameters.  
>- The value of the "motionSensitivity" parameter can be one of: {low, medium, high}. As the name implies, it sets the sensitivity of the motion detection processor.
>- The value of the "grpcAIServerAddress" parameter points to the address of the inference server. The name of the module that we developed in this sample and deployed in our module deployment manifest was set to "isServer", which was listening to an inference request on "/score" endpoint through port 44001. Here, we set the exact address of the scoring endpoint.
>- The "hubSinkOutputName" parameter sets the IoT Hub channel name which ingests inference results into IoT Hub.

In [ ]:
mediaGraphTopologyParameters = {
          "name": "Sample-Graph-1",
          "properties": {
            "topologyName": "SampleTopology",
            "description": "Sample graph description",
            "parameters": [
              {
                "name": "rtspUrl",
                "value": "rtsp://rtspsim:554/media/sample.mkv"
              },
              {
                "name": "rtspUsername",
                "value": "username"
              },
              {
                "name": "rtspPassword",
                "value": "password"
              },
              {
                "name": "motionSensitivity",
                "value": "medium"
              },
              {
                "name": "httpAIServerAddress",
                "value": "http://isServer:5001/score"
              },
              {
                "name": "hubSinkOutputName",
                "value": "inferences"
              }
            ]
          }
        }

After setting the parameters above (as Python dictionary data structure), we can now set an instance of the previously deployed topology on the Edge device with the custom parameters above.

In [ ]:
# Set topology instance
graphManager.GenericCall("GraphInstanceSet", mediaGraphTopologyParameters)
print(response)

#### 9.1.3.6. Activate Topology Instance
Next, we activate the Topology Instance that we set in the previous node.

>[!IMPORTANT]  
>Be sure to set the name parameter below to the exact topology instance name that we used in the previous node.

In [ ]:
# Activate topology instance
operationParams = {
                    "name": "Sample-Graph-1"
                    }

graphManager.GenericCall("GraphInstanceActivate", operationParams)
print(response)

#### 9.1.3.7. De-Activate Topology Instance
In case you do not want to run the topology, you can de-activate it at any time. The following code snippet will de-activate your instance, meaning it will no longer be running.

>[!IMPORTANT]  
>Be sure to set the name parameter below to the exact topology instance name that we used in the previous node.

In [ ]:
# De-Activate topology instance
operationParams = {
                    "name": "Sample-Graph-1"
                    }

graphManager.GenericCall("GraphInstanceDeactivate", operationParams)
print(response)

#### 9.1.3.8. Delete Graph Topology Instance
The following code snippet is shows how to delete a previously deployed instance.

In [ ]:
# Delete instance
operationParams = {
                    "name": "Sample-Graph-1"
                    }

graphManager.GenericCall("GraphInstanceDelete", operationParams)
print(response)

#### 9.1.3.9. Delete Graph Topology
The following code snippet shows how to delete the previously deployed toplogy.

>[!IMPORTANT]  
>Be sure to set the name parameter below to the exact topology name that you want to delete.

In [ ]:
# Delete topology
operationParams = {
                    "name": "SampleTopology"
                    }

graphManager.GenericCall("GraphTopologyDelete", operationParams)
print(response)

## 9.2. Monitoring the Output of the LVA Edge Module

In the sample media graph topology, we defined two "sink" nodes:  
* Sink 1 - sends the inference result (Json payload) to IoT Hub
* Sink 2 - sends the stream chunks (where motion detected) into Azure Media Services

### 9.2.1. Monitoring IoT Edge Module and IoT Hub Message Flow
In VSCode, under the "AZURE IOT HUB" panel,  
1. Right click on the device name.  
2. Click on "Start Monitoring Built-In Event Endpoint".  


<img src="documents/_monitor01.png" width=500px/>   


3. You will see the message flow with inference results and the video chunk logs that are sent to Azure Media Services cloud service.  


<img src="documents/_monitor02.png" width=1200px/>   

### 9.2.2. Monitoring the archived video streams in Azure Media Services
On the Azure portal, explore the resource group used to create LVA and open the Azure Media Services resource that we created earlier in this sample. In the "assets" section of Azure Media Services, you will see chunks of recording where motion was detected.


<img src="documents/_monitor03.png" width=1000px/>   


To play these clips, click on the desired asset. Then, locate the "Streaming URL" textbox and click on the "Create new" link. In the pane that opens for "Add streaming locator", accept the defaults and hit "Add" at the bottom. In the Asset details page, the video player should now load to the first frame of the video, and you can hit the play button.


<img src="documents/asset-streaming-url.png" width=400px/>   